

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/NER_HUMAN_PHENOTYPE_GENE_CLINICAL.ipynb)




# **Detect genes and human phenotypes**

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload `license_keys.json` to the folder that opens.
Otherwise, you can look at the example outputs at the bottom of the notebook.



## 1. Colab Setup

In [ ]:
import json, os
from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

In [3]:
import sparknlp
import sparknlp_jsl

from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_colwidth', 200)

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print("Spark NLP Version :", sparknlp.version())
print("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark

Spark NLP Version : 4.2.8
Spark NLP_JSL Version : 4.2.8


## 2. Select the NER model and construct the pipeline

### Select the NER model

For more details: https://github.com/JohnSnowLabs/spark-nlp-models#pretrained-models---spark-nlp-for-healthcare

In [4]:
MODEL_NAME = "ner_human_phenotype_gene_clinical"

### Create the pipeline

In [5]:
document_assembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

sentence_detector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence","token"])\
  .setOutputCol("embeddings")

clinical_ner = MedicalNerModel.pretrained(MODEL_NAME, "en", "clinical/models")\
  .setInputCols(["sentence", "token", "embeddings"])\
  .setOutputCol("ner_label")

ner_converter = NerConverterInternal()\
  .setInputCols(["sentence", "token", "ner_label"])\
  .setOutputCol("ner_chunk")

ner_pipeline = Pipeline(stages=[document_assembler,
                                sentence_detector,
                                tokenizer,
                                embeddings,
                                clinical_ner,
                                ner_converter])


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_human_phenotype_gene_clinical download started this may take some time.
[OK!]


## 3. Create example inputs

In [6]:
# Enter examples as strings in this array
text_list = ["""We will systematically examine seven genes (CHN1, MDH1, PCP4, RTN1, SLC14A1, SNAP25 and VSNL1) that are altered in the three neurodegenerative diseases.""",
             """The eye features were a notable part of this child's presentation and mutations in the linkeropathy genes (XYLT1, XYLT2, B4GALT7, B3GALT6, and B3GAT3) can be associated with ocular findings, including corneal clouding, strabismus, nystagmus, cataracts, glaucoma, and retinal abnormalities, including retinal detachment.""",
             """Previous genome-wide association studies have identified several genes associated with obesity, including LEP, POMC, PCSK1, FTO, GNPDA2, TMEM18, BDNF, ETV5, SEC16B, and  SIM1.""",
             """Different corneal dystrophies are caused by mutations in the CHST6, KRT3, KRT12, PIP5K3, SLC4A11, TACSTD2, TGFBI, and UBIAD1 genes.""",
             """Some genes, such as GSTM1, DNMT3L, and CYP1A1, have been shown to be potentially associated with oligospermia; some, such as CATSPER1, CRISP2, SEPT4, TCTE3, TEKT4, and DNAH1 with asthenospermia; and still others, such as DPY19L2 and AURKC, with teratospermia."""
             ]

## 4. Use the pipeline to create outputs

In [7]:
from pyspark.sql.types import StringType, IntegerType

df = spark.createDataFrame(text_list,StringType()).toDF('text')
result = ner_pipeline.fit(df).transform(df)

result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|          embeddings|           ner_label|           ner_chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|We will systemati...|[{document, 0, 15...|[{document, 0, 15...|[{token, 0, 1, We...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 44, 47, ...|
|The eye features ...|[{document, 0, 31...|[{document, 0, 31...|[{token, 0, 2, Th...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 107, 111...|
|Previous genome-w...|[{document, 0, 17...|[{document, 0, 17...|[{token, 0, 7, Pr...|[{word_embeddings...|[{named_entity, 0...|[{chunk, 87, 93, ...|
|Different corneal...|[{document, 0, 13...|[{document, 0, 13...|[{token, 0, 8, Di...|[{word_embeddings...|

In [8]:
result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                     result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(50,truncate=False)

+--------------------------+---------+
|chunk                     |ner_label|
+--------------------------+---------+
|CHN1                      |GENE     |
|MDH1                      |GENE     |
|PCP4                      |GENE     |
|RTN1                      |GENE     |
|SLC14A1                   |GENE     |
|SNAP25                    |GENE     |
|VSNL1                     |GENE     |
|neurodegenerative diseases|HP       |
|XYLT1                     |GENE     |
|XYLT2                     |GENE     |
|B4GALT7                   |GENE     |
|B3GALT6                   |GENE     |
|B3GAT3                    |GENE     |
|corneal clouding          |HP       |
|strabismus                |HP       |
|nystagmus                 |HP       |
|cataracts                 |HP       |
|glaucoma                  |HP       |
|retinal abnormalities     |HP       |
|retinal detachment        |HP       |
|obesity                   |HP       |
|LEP                       |GENE     |
|POMC                    

## 5. Visualize results

In [9]:
from sparknlp_display import NerVisualizer

ner_vis = NerVisualizer()

for i in range(len(text_list)):
    ner_vis.display(result = result.collect()[i], label_col = "ner_chunk", document_col="document")
    print("\n\n\n")